In [34]:
import numpy as np
import pandas as pd

In [35]:
#import files
#read the txt file
with open('spa.txt') as f:
  text=f.read()
print(text[:50])

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.



In [37]:
data=text.split('\n')
data=[i.split('\t') for i in data]


In [40]:
df=pd.DataFrame(data,columns=['English','Spanish'])

length=len(data)
df=df.sample(length//2)
df.head()

,English,Spanish
100962,It's good to try new things once in a while.,A veces es bueno probar cosas nuevas.
113093,I want to be sure you understand what's going ...,Quiero estar seguro de que entiendes lo que va...
23948,I let go of the rope.,Solté la cuerda.
52103,He said he didn't like this.,Él dijo que no le gustaba esto.
89661,We'll be safe as long as we stay here.,Vamos a estar seguros mientras nos quedemos aquí.


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59482 entries, 100962 to 14574
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  59482 non-null  object
 1   Spanish  59481 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [42]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True) # Add this line

In [43]:
df.sample(5)

,English,Spanish
6762,That's why I told you not to go by yourself.,Por eso te dije que no fueras solo.
37262,Tom looks worried.,Tomás parece preocupado.
7267,No one will change anything.,Nadie cambiará nada.
44986,What conclusions did they come to?,¿A qué conclusiones llegaron?
46481,Please tell me about your problem.,"Por favor, háblame de tu problema."


In [44]:
df['English']=df['English'].astype(str)
df['Spanish']=df['Spanish'].astype(str)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59482 entries, 0 to 59481
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  59482 non-null  object
 1   Spanish  59482 non-null  object
dtypes: object(2)
memory usage: 929.5+ KB


In [46]:
df['English'] = df['English'].fillna('').apply(lambda x: x.lower())
df['Spanish'] = df['Spanish'].fillna('').apply(lambda x: x.lower())
# df['English'] = df['English'].str.replace('.', '') # Original
# df['Spanish'] = df['Spanish'].str.replace('.', '') # Original
df['English'] = df['English'].str.replace(r'\.', '', regex=True) # Changed
df['Spanish'] = df['Spanish'].str.replace(r'\.', '', regex=True) # Changed

# Tokenization

In [47]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 115.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 112.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [48]:
import spacy

spacy_en = spacy.load("en_core_web_sm")
spacy_es = spacy.load("es_core_news_sm")


In [49]:
def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_es(text):
    return [tok.text.lower() for tok in spacy_es.tokenizer(text)]

In [50]:
df['English']=df['English'].apply(tokenize_en)
df['Spanish']=df['Spanish'].apply(tokenize_es)

In [51]:
df.sample(5)

,English,Spanish
36169,"[i, keep, thinking, about, tom]","[me, mantengo, pensando, en, tom]"
3310,"[ca, n't, he, ride, a, bicycle, ?]","[¿, no, sabe, montar, en, bicicleta, ?]"
47189,"[you, 've, gained, weight, ,, have, n't, you, ?]","[has, aumentado, de, peso, ,, ¿, no, es, verda..."
25286,"[i, 'm, not, sure, what, i, should, do]","[no, estoy, segura, de, qué, debería, hacer]"
43563,"[tom, can, go, hear, mary, sing, at, that, clu...","[tom, puede, ir, a, escuchar, cantar, a, mary,..."


In [52]:

df['Spanish'] = df['Spanish'].apply(lambda x: ['<sos>'] + x + ['<eos>'])

In [17]:
df.sample(5)

,English,Spanish
100250,"[a, few, glasses, of, wine, can, loosen, the, ...","[<sos>, unos, vasos, de, vino, pueden, soltar,..."
12792,"[did, tom, know, mary, ?]","[<sos>, ¿, conocía, tom, a, mary, ?, <eos>]"
12774,"[come, sing, with, me]","[<sos>, ven, a, cantar, conmigo, <eos>]"
17072,"[it, rained, for, days]","[<sos>, llovió, durante, días, <eos>]"
53897,"[she, wants, to, meet, him, again]","[<sos>, ella, quiere, volver, a, encontrarse, ..."


In [53]:
spalen=0
for x in df['Spanish']:
  if len(x)>spalen:
    spalen=len(x)
print(spalen)

englen=0
for x in df['English']:
  if len(x)>englen:
    englen=len(x)
print(englen)

52
48


In [54]:
df['English'] = df['English'].apply(lambda x: x + ['<pad>'] * (englen - len(x)))
df['Spanish'] = df['Spanish'].apply(lambda x: x + ['<pad>'] * (spalen - len(x)))

In [55]:
engword = df['English'].explode().unique()
spaword = df['Spanish'].explode().unique()

In [56]:
eng_tok_stoi = dict([(char, i) for i,char in enumerate(engword)])
spa_tok_stoi = dict([(char, i) for i,char in enumerate(spaword)])

eng_tok_itos = dict([(i,char)for i ,char in enumerate(engword)])
spa_tok_itos = dict([(i,char)for i ,char in enumerate(spaword)])

In [57]:
df['English']=df['English'].apply(lambda x: [eng_tok_stoi[word] for word in x])
df['Spanish']=df['Spanish'].apply(lambda x: [spa_tok_stoi[word] for word in x])

In [58]:
len(eng_tok_itos)

10431

In [24]:
# import random
# import torch
# import torch.nn as nn

# class Encoder(nn.Module):
#     def __init__(self, input_dim, emb_dim, hid_dim,num_layer, dropout):
#         super().__init__()
#         self.hid_dim = hid_dim
#         self.embedding = nn.Embedding(input_dim, emb_dim)
#         self.lstm = nn.LSTM(emb_dim, hid_dim,num_layer,dropout=dropout)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, src):
#         embedded = self.dropout(self.embedding(src))
#         outputs, (hidden, cell) = self.lstm(embedded)
#         return outputs, hidden, cell

# class Decoder(nn.Module):
#     def __init__(self,input_dim,  emb_dim,output_dim, hid_dim,num_layers, dropout):
#         super().__init__()

#         self.hid_dim = hid_dim
#         self.output_dim = output_dim

#         self.embedding = nn.Embedding(output_dim, emb_dim)
#         self.lstm = nn.LSTM(emb_dim, self.hid_dim, num_layers=num_layers, dropout=dropout)
#         self.fc_out = nn.Linear(self.hid_dim, output_dim)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, input, hidden, cell):

#         input = input.unsqueeze(0)
#         # input = [1, batch size]

#         embedded = self.dropout(self.embedding(input))
#         # embedded = [1, batch size, emb dim]

#         # hidden = hidden.unsqueeze(0)
#         # cell = cell.unsqueeze(0)
#         output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

#         # seq len and n layers/directions will always be 1 in this decoder, so we can squeeze
#         embedded = embedded.squeeze(0)
#         output = output.squeeze(0)
#         # hidden = hidden.squeeze(0)
#         # cell = cell.squeeze(0)


#         prediction = self.fc_out(output)
#         # prediction = [batch size, output dim]

#         return prediction, hidden, cell


# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder, device):
#         super().__init__()

#         self.encoder = encoder
#         self.decoder = decoder
#         self.device = device


#     def forward(self, src, trg, teacher_forcing_ratio=0.5):

#         batch_size = trg.shape[1]
#         trg_len = trg.shape[0]
#         trg_vocab_size = self.decoder.output_dim

#         outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

#         encoder_outputs, hidden, cell = self.encoder(src)
#         hidden = hidden[:, :batch_size, :]
#         cell = cell[:, :batch_size, :]
#         input = trg[1, :]

#         for t in range(1, trg_len):

#             output, hidden, cell = self.decoder(input, hidden, cell)

#             outputs[t] = output

#             teacher_force = random.random() < teacher_forcing_ratio


#             top1 = output.argmax(1)


#             input = trg[t] if teacher_force else top1

#         return outputs

# INPUT_DIM = len(eng_tok_itos)
# OUTPUT_DIM = len(spa_tok_itos)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# HID_DIM = 512
# ENC_DROPOUT = 0.2
# DEC_DROPOUT = 0.2
# N_LAYERS=2

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS ,ENC_DROPOUT)
# decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, OUTPUT_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

# model = Seq2Seq(encoder, decoder, device).to(device)
# print(model)

In [25]:
pad_idx = spa_tok_stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(model.parameters())

NameError: name 'nn' is not defined

In [59]:
train_len = int(len(df)*0.8)
train_len

47585

In [60]:
X_train = df['English'].iloc[:int(train_len)]
y_train = df['Spanish'].iloc[:int(train_len)]
X_test = df['English'].iloc[int(train_len):]
y_test = df['Spanish'].iloc[int(train_len):]

In [61]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, english_data, spanish_data):
        self.english_data = english_data
        self.spanish_data = spanish_data

    def __len__(self):
        return len(self.english_data)

    def __getitem__(self, idx):
        english_seq = torch.tensor(self.english_data.iloc[idx], dtype=torch.long)
        spanish_seq = torch.tensor(self.spanish_data.iloc[idx], dtype=torch.long)
        return english_seq, spanish_seq

# Assuming you have already created your df, englen, spalen, eng_tok_index, spa_tok_index

# Create Dataset instances
train_dataset = TranslationDataset(X_train, y_train)
test_dataset = TranslationDataset(X_test, y_test)

# Create DataLoader instances
batch_size = 32 # You can adjust this
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [62]:

for eng_seq, spa_seq in train_dataloader:
  print("English Sequence Batch Shape:", eng_seq.shape)
  print("Spanish Sequence Batch Shape:", spa_seq.shape)
  break

English Sequence Batch Shape: torch.Size([32, 48])
Spanish Sequence Batch Shape: torch.Size([32, 52])


In [63]:
# import random
# import torch
# import torch.nn as nn

# class Encoder(nn.Module):
#     def __init__(self, input_dim, emb_dim, hid_dim,num_layer, dropout):
#         super().__init__()
#         self.hid_dim = hid_dim
#         self.embedding = nn.Embedding(input_dim, emb_dim)
#         self.lstm = nn.LSTM(emb_dim, hid_dim,num_layer,dropout=dropout)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, src):
#         # src = [src_len, batch_size]

#         embedded = self.dropout(self.embedding(src))
#         # embedded = [src_len, batch_size, emb_dim]

#         outputs, (hidden, cell) = self.lstm(embedded)
#         # outputs = [src_len, batch_size, hid_dim * num_directions]
#         # hidden = [num_layers * num_directions, batch_size, hid_dim]
#         # cell = [num_layers * num_directions, batch_size, hid_dim]

#         return outputs, hidden, cell

# class Decoder(nn.Module):
#     def __init__(self,input_dim,  emb_dim,output_dim, hid_dim,num_layers, dropout):
#         super().__init__()

#         self.hid_dim = hid_dim
#         self.output_dim = output_dim

#         self.embedding = nn.Embedding(output_dim, emb_dim)
#         self.lstm = nn.LSTM(emb_dim, self.hid_dim, num_layers=num_layers, dropout=dropout)
#         self.fc_out = nn.Linear(self.hid_dim, output_dim)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, input, hidden, cell):
#         # input = [batch size]
#         # hidden = [num_layers * num_directions, batch size, hid_dim]
#         # cell = [num_layers * num_directions, batch size, hid_dim]

#         # reshape input to be 1 token per batch for the decoder LSTM
#         input = input.unsqueeze(0)
#         # input = [1, batch size]

#         embedded = self.dropout(self.embedding(input))
#         # embedded = [1, batch size, emb dim]

#         # Pass through LSTM
#         # output = [seq_len, batch_size, hid_dim * num_directions]
#         # hidden = [num_layers * num_directions, batch size, hid_dim]
#         # cell = [num_layers * num_directions, batch size, hid_dim]
#         output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

#         # seq len and n layers/directions will always be 1 in this decoder, so we can squeeze
#         output = output.squeeze(0)
#         # output = [batch size, hid_dim]

#         prediction = self.fc_out(output)
#         # prediction = [batch size, output dim]

#         return prediction, hidden, cell


# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder, device):
#         super().__init__()

#         self.encoder = encoder
#         self.decoder = decoder
#         self.device = device


#     def forward(self, src, trg, teacher_forcing_ratio=0.5):
#         # src = [src_len, batch_size]
#         # trg = [trg_len, batch_size]

#         trg_len = trg.shape[0]
#         # Get the actual batch size from the input tensor
#         batch_size = trg.shape[1] # Or src.shape[1], they should be the same

#         trg_vocab_size = self.decoder.output_dim

#         # Tensor to store decoder outputs
#         outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

#         # Encode source sequence
#         encoder_outputs, hidden, cell = self.encoder(src)

#         # Initial input to the decoder is the <sos> token
#         input = trg[0, :] # Use the first token of the target sequence

#         for t in range(1, trg_len):
#             # Pass through decoder
#             # input = [batch size]
#             # hidden = [num_layers * num_directions, batch size, hid_dim]
#             # cell = [num_layers * num_directions, batch size, hid_dim]
#             output, hidden, cell = self.decoder(input, hidden, cell)
#             # output = [batch size, output_dim]
#             # hidden = [num_layers * num_directions, batch size, hid_dim]
#             # cell = [num_layers * num_directions, batch size, hid_dim]


#             # Store prediction in outputs tensor
#             outputs[t] = output

#             # Decide if we are going to use teacher forcing or not
#             teacher_force = random.random() < teacher_forcing_ratio

#             # Get the highest predicted token from the model output
#             top1 = output.argmax(1)

#             # If teacher forcing, use the actual next token as input
#             # If not teacher forcing, use the model's predicted token
#             input = trg[t] if teacher_force else top1

#         return outputs

# # --- Rest of your code ---
# INPUT_DIM = len(eng_tok_itos)
# OUTPUT_DIM = len(spa_tok_itos)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# HID_DIM = 512
# ENC_DROPOUT = 0.2
# DEC_DROPOUT = 0.2
# N_LAYERS=2

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS ,ENC_DROPOUT)
# decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, OUTPUT_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

# model = Seq2Seq(encoder, decoder, device).to(device)
# print(model)

# pad_idx = spa_tok_stoi['<pad>']
# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
# optimizer = torch.optim.Adam(model.parameters())

# train_len = int(len(df)*0.8)
# # print(train_len) # Optional: check the train length

# X_train = df['English'].iloc[:int(train_len)]
# y_train = df['Spanish'].iloc[:int(train_len)]
# X_test = df['English'].iloc[int(train_len):]
# y_test = df['Spanish'].iloc[int(train_len):]

# class TranslationDataset(Dataset):
#     def __init__(self, english_data, spanish_data):
#         self.english_data = english_data
#         self.spanish_data = spanish_data

#     def __len__(self):
#         return len(self.english_data)

#     def __getitem__(self, idx):
#         # Ensure data is already in list of token indices format
#         english_seq = torch.tensor(self.english_data.iloc[idx], dtype=torch.long)
#         spanish_seq = torch.tensor(self.spanish_data.iloc[idx], dtype=torch.long)
#         return english_seq, spanish_seq

# # Assuming you have already created your df, englen, spalen, eng_tok_index, spa_tok_index

# # Create Dataset instances
# train_dataset = TranslationDataset(X_train, y_train)
# test_dataset = TranslationDataset(X_test, y_test)

# # Create DataLoader instances
# batch_size = 32 # You can adjust this
# # Add collate_fn to handle padding if necessary, although you padded already
# # If you didn't pad before DataLoader, you would need a collate_fn here
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # Optional: Check a batch shape
# # for eng_seq, spa_seq in train_dataloader:
# #   print("English Sequence Batch Shape:", eng_seq.shape) # Should be [batch_size, seq_len]
# #   print("Spanish Sequence Batch Shape:", spa_seq.shape) # Should be [batch_size, seq_len]
# #   break

# def train(model, dataloader, optimizer, criterion, clip):
#     model.train()

#     epoch_loss = 0

#     for src, trg in dataloader:
#         # DataLoaders return [batch_size, seq_len]. RNN expects [seq_len, batch_size]
#         src = src.transpose(0, 1).to(device) # [src_len, batch_size]
#         trg = trg.transpose(0, 1).to(device) # [trg_len, batch_size]

#         optimizer.zero_grad()

#         output = model(src, trg)
#         # output shape: [trg_len, batch_size, output_dim]

#         # Reshape output and trg for loss calculation:
#         # Ignore first token (usually <sos>)
#         output_dim = output.shape[-1]
#         output = output[1:].view(-1, output_dim)  # [(trg_len-1)*batch_size, output_dim]
#         trg = trg[1:].reshape(-1)                    # [(trg_len-1)*batch_size]

#         loss = criterion(output, trg)
#         loss.backward()

#         # Gradient clipping (to avoid exploding gradients)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

#         optimizer.step()

#         epoch_loss += loss.item()

#     return epoch_loss / len(dataloader)


# def evaluate(model, dataloader, criterion):
#     model.eval()

#     epoch_loss = 0

#     with torch.no_grad():
#         for src, trg in dataloader:
#             # DataLoaders return [batch_size, seq_len]. RNN expects [seq_len, batch_size]
#             src = src.transpose(0, 1).to(device)
#             trg = trg.transpose(0, 1).to(device)

#             output = model(src, trg, teacher_forcing_ratio=0)  # no teacher forcing

#             output_dim = output.shape[-1]
#             output = output[1:].view(-1, output_dim)
#             trg = trg[1:].reshape(-1)

#             loss = criterion(output, trg)
#             epoch_loss += loss.item()

#     return epoch_loss / len(dataloader)


# N_EPOCHS = 10
# CLIP = 1

# for epoch in range(N_EPOCHS):
#     train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
#     val_loss = evaluate(model, test_dataloader, criterion)

#     print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}')

In [64]:

import random
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, num_layer, dropout): # num_layer is fine as a var name
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # num_layer is passed as the 3rd positional arg to LSTM, which is 'num_layers'
        self.lstm = nn.LSTM(emb_dim, hid_dim, num_layer, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src_len, batch_size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src_len, batch_size, emb_dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [src_len, batch_size, hid_dim * num_directions]
        # hidden = [num_layers * num_directions, batch_size, hid_dim]
        # cell = [num_layers * num_directions, batch_size, hid_dim]
        return outputs, hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_dim, emb_dim, output_dim, hid_dim, num_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim) # output_dim for embedding size
        self.lstm = nn.LSTM(emb_dim, self.hid_dim, num_layers=num_layers, dropout=dropout)
        self.fc_out = nn.Linear(self.hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [num_layers * num_directions, batch size, hid_dim]
        # cell = [num_layers * num_directions, batch size, hid_dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, emb dim]
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output = [seq_len=1, batch_size, hid_dim * num_directions]
        # hidden = [num_layers * num_directions, batch size, hid_dim]
        # cell = [num_layers * num_directions, batch size, hid_dim]
        output = output.squeeze(0) # Remove seq_len dimension
        # output = [batch size, hid_dim]
        prediction = self.fc_out(output)
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src = [src_len, batch_size]
        # trg = [trg_len, batch_size]
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden, cell = self.encoder(src)

        # Initial input to the decoder is the <sos> token from the target sequence
        input = trg[0, :] # Corrected: Use the first token of target

        for t in range(1, trg_len): # Start from 1 because trg[0] is input, predict trg[1]
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) # Get most likely next token
            input = trg[t] if teacher_force else top1 # Use ground truth or predicted token for next input
        return outputs

# Hyperparameters and Instantiation (moved from cell 31 for clarity)
INPUT_DIM = len(eng_tok_itos)
OUTPUT_DIM = len(spa_tok_itos)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
N_LAYERS = 2 # This is num_layer for Encoder and num_layers for Decoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
# For Decoder, input_dim is not used for the first layer, output_dim is used for embedding
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, OUTPUT_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(encoder, decoder, device).to(device)
print(model)



Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10431, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(19311, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.2)
    (fc_out): Linear(in_features=512, out_features=19311, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)


In [65]:
import torch.nn as nn
pad_idx = spa_tok_stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(model.parameters())

In [69]:

class TranslationDataset(Dataset):
    def __init__(self, english_data, spanish_data):
        self.english_data = english_data
        self.spanish_data = spanish_data

    def __len__(self):
        return len(self.english_data)

    def __getitem__(self, idx):
        english_seq = torch.tensor(self.english_data.iloc[idx], dtype=torch.long)
        spanish_seq = torch.tensor(self.spanish_data.iloc[idx], dtype=torch.long)
        return english_seq, spanish_seq

train_dataset = TranslationDataset(X_train, y_train)
test_dataset = TranslationDataset(X_test, y_test)

batch_size = 512 # You can adjust this
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train and evaluate functions (from original cell 31, these are fine)
def train(model, dataloader, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for src, trg in dataloader:
        src = src.transpose(0, 1).to(device)
        trg = trg.transpose(0, 1).to(device)
        optimizer.zero_grad()
        output = model(src, trg) # teacher_forcing_ratio defaults to 0.5
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        print(epoch_loss)
    return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in dataloader:
            src = src.transpose(0, 1).to(device)
            trg = trg.transpose(0, 1).to(device)
            output = model(src, trg, teacher_forcing_ratio=0) # Turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            print(epoch_loss)
    return epoch_loss / len(dataloader)

# Training loop (from original cell 31, this is fine)
N_EPOCHS = 1
CLIP = 1

print("Starting training...")
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    val_loss = evaluate(model, test_dataloader, criterion)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}')

Starting training...
5.2779316902160645
10.416745662689209
15.650460243225098
20.656099796295166
25.930034637451172
31.193334579467773
36.39363241195679
41.63727378845215
46.92459297180176
52.24046182632446
57.441341400146484
62.65782690048218
67.7383770942688
72.96782684326172
78.2213249206543
83.42110347747803
88.50654363632202
93.71275568008423
98.86668062210083
104.00850486755371
109.3213210105896
114.45543622970581
119.58595943450928
124.66270351409912
130.00143146514893
135.2685432434082
140.42535734176636
145.62556266784668
150.73276948928833
155.92762660980225
160.9737777709961
166.1340208053589
171.17666482925415
176.2986044883728
181.34061861038208
186.69626998901367
191.89312314987183
196.98477029800415
202.08680486679077
207.2999587059021
212.46444034576416
217.6364803314209
222.62923336029053
227.62763214111328
232.76050853729248
237.9291796684265
243.07232332229614
248.24383449554443
253.2020378112793
258.32879734039307
263.3948287963867
268.504647731781
273.4636626243591

In [71]:

def translate_sentence(sentence, model, eng_tokenizer, spa_tokenizer, englen, spalen, device, spa_tok_itos, spa_tok_stoi):
    model.eval() # Set model to evaluation mode

    # Tokenize and preprocess the input sentence
    tokens = eng_tokenizer(sentence.lower())
    # Add padding
    if len(tokens) < englen:
        tokens = tokens + ['<pad>'] * (englen - len(tokens))
    elif len(tokens) > englen:
        tokens = tokens[:englen] # Truncate if too long

    # Convert tokens to indices
    indexed_tokens = [eng_tok_stoi.get(token, eng_tok_stoi['<pad>']) for token in tokens] # Handle OOV tokens with pad_idx

    # Convert to tensor and add batch dimension (batch_size=1)
    src_tensor = torch.LongTensor(indexed_tokens).unsqueeze(1).to(device) # [src_len, 1]

    with torch.no_grad():
        # Encode the source sentence
        encoder_outputs, hidden, cell = model.encoder(src_tensor)

    # Initial input to the decoder is the <sos> token
    # Ensure <sos> token exists in spa_tok_stoi
    if '<sos>' not in spa_tok_stoi:
        raise ValueError("<sos> token not found in Spanish vocabulary.")
    input_token = torch.LongTensor([spa_tok_stoi['<sos>']]).to(device) # [1]

    translated_tokens = []

    # Max target sequence length (should be consistent with spalen)
    # We predict up to spalen-1 tokens because the first token is <sos>
    for t in range(spalen):
        # Pass through decoder
        output, hidden, cell = model.decoder(input_token, hidden, cell)
        # output shape: [1, output_dim] (batch_size=1)

        # Get the predicted next token
        pred_token_index = output.argmax(1).item()

        # Convert index back to token
        pred_token = spa_tok_itos[pred_token_index]

        # Add to translated sequence
        translated_tokens.append(pred_token)

        # If the predicted token is <eos>, stop decoding
        if pred_token == '<eos>':
            break

        # Set the predicted token as the input for the next step
        input_token = torch.LongTensor([pred_token_index]).to(device) # [1]
    filtered_tokens = [token for token in translated_tokens if token not in ['<sos>', '<eos>', '<pad>']]

    # Join tokens to form the translated sentence
    translated_sentence = " ".join(filtered_tokens)

    return translated_sentence


# Example sentence to translate
input_english_sentence = "how are you"

# Call the translation function
translated_spanish_sentence = translate_sentence(
    input_english_sentence,
    model,
    tokenize_en, # Pass the English tokenizer function
    tokenize_es, # Pass the Spanish tokenizer function (not used directly in translate_sentence, but good practice to have it)
    englen,
    spalen,
    device,
    spa_tok_itos,
    spa_tok_stoi
)

print(f"English: {input_english_sentence}")
print(f"Spanish: {translated_spanish_sentence}")

# Translate another sentence
input_english_sentence_2 = "i like this"
translated_spanish_sentence_2 = translate_sentence(
    input_english_sentence_2,
    model,
    tokenize_en,
    tokenize_es,
    englen,
    spalen,
    device,
    spa_tok_itos,
    spa_tok_stoi
)

print(f"English: {input_english_sentence_2}")
print(f"Spanish: {translated_spanish_sentence_2}")


English: how are you
Spanish: ¿ te gusta ?
English: i like this
Spanish: me que
